In [18]:
import spacy
import numpy as np 
import pandas as pd
import ujson
import sys
import time
import pickle

from tqdm import tqdm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker, CandidateGenerator
from scispacy.umls_utils import UmlsKnowledgeBase
# from bigbio.dataloader import BigBioConfigHelpers
from typing import List, Optional
from collections import defaultdict
from matplotlib import pyplot as plt

import pubmed_parser as pp

In [2]:
nlp_lg = spacy.load("en_core_sci_lg")
nlp_lg.add_pipe("abbreviation_detector")
nlp_lg.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls", 'filter_for_definitions':False, "max_entities_per_mention":100, "no_definition_threshold":.7, "threshold":0.4})

nlp_sm = spacy.load('en_core_sci_sm')
nlp_sm.add_pipe("abbreviation_detector")
nlp_sm.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls", 'filter_for_definitions':False, "max_entities_per_mention":20, "no_definition_threshold":.95, "threshold":0.9})

/nethome/dkartchner3/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/nethome/dkartchner3/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [19]:
nlp_sm = spacy.load('en_core_sci_sm')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/nethome/dkartchner3/conda_envs/contrastive_el/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_16186/3572589338.py", line 1, in <module>
    nlp_sm = spacy.load('en_core_sci_sm')
  File "/nethome/dkartchner3/conda_envs/contrastive_el/lib/python3.10/site-packages/spacy/__init__.py", line 51, in load
    keyed by section values in dot notation.
  File "/nethome/dkartchner3/conda_envs/contrastive_el/lib/python3.10/site-packages/spacy/util.py", line 420, in load_model
    """
  File "/nethome/dkartchner3/conda_envs/contrastive_el/lib/python3.10/site-packages/spacy/util.py", line 453, in load_model_from_package
    name (str): The package name.
  File "/nethome/dkartchner3/conda_envs/contrastive_el/lib/python3.10/site-packages/en_core_sci_sm/__init__.py", line 10, in load
    return load_model_from_init_py(__file__, **overrides)
  Fil

In [14]:
def process_abstract_scispacy(text: str,
                    model,
                    eps: float=1e-6, 
                    db_name: str='UMLS',
                    thresh=0.99):
    '''
    SciSpacy processing of a single abstract

    We only include confident entities
    '''
    doc = model(text)
    ent_list = []
    for ent in doc.ents:
        offsets = [(ent.start_char, ent.end_char)]
        text = ent.text
        candidates = [e[0] for e in ent._.kb_ents if e[1] > thresh]

        if len(candidates) != 1:
            continue
        
        ent_list.append({
            "offsets":offsets, 
            "cui": candidates[0], 
            })
    return ent_list

In [5]:
dict_outputs = pp.parse_medline_xml('/mitchell/nlp_corpora/pubmed/pubmed23n0001.xml.gz')
len(dict_outputs)

In [11]:
abstract_chunk = dict_outputs

In [9]:
filtered_abstracts = [x for x in dict_outputs if len(x['abstract']) > 50]
len(filtered_abstracts)

15377

In [15]:
all_processed_abstracts = []
excluded_pmids = []
i = 0
pmid_to_ind = {}
pmcid_to_ind = {}

for a in tqdm(abstract_chunk):
    if len(a['abstract']) < 50:
        excluded_pmids.append(a['pmid'])
        continue
    pmid_to_ind[a['pmid']] = i
    title = a['title'].strip()
    abstract = a['abstract'].strip()
    text = title + '\n' + abstract
    entities = process_abstract_scispacy(text, nlp_lg)
    output = {
        'pmid': a['pmid'],
        'title': title,
        'abstract': abstract,
        'entities': entities,
    }
    all_processed_abstracts.append(output)
    i += 1


100%|██████████| 30000/30000 [17:25<00:00, 28.70it/s]  
